# Practice 2. ConvLab을 이용한 Pipelined 대화 시스템 구축

# 목차

## Step 1. MultiWOZ 데이터셋을 살펴보자

## Step 2. ConvLab을 활용해 Pipelined 대화 시스템을 구축하자

## Step 3. ConvLab에서 제공하는 모듈들로 모델들을 구성 및 진단하고, 평가하자

## Additional. End-to-end Neural Pipeline (ACL 2020) 모델을 사용해보자

# Step 1. MultiWOZ 데이터셋을 살펴보자

## Step 1.0 필요한 module들을 정의합니다

아래 코드를 도와주는 module들 이며, 필요시 수정할 수 있습니다.

In [ ]:
import json
import os
import zipfile
import sys
from collections import Counter
from nltk.tokenize import word_tokenize

from textwrap import indent
from pprint import pformat
from pprint import pprint

def read_zipped_json(filepath, filename):
    archive = zipfile.ZipFile(filepath, 'r')
    return json.load(archive.open(filename))


def pprint_manual(user_manual, name):
    """print user manual
        argument 'name' is needed to discriminate 'WOZ' from others
    """
    print('    User manual (message) : ')
    if 'WOZ' in name:
        print(" "*8, user_manual)
    else:
        for manual_one in user_manual:
            print(" "*8, manual_one)


def pprint_goal(goal, name):
    """print user's goal
        argument 'name' is needed to discriminate "WOZ" from others.
    """
    if 'WOZ' in name:
        pass
    else:
        for i, mes in enumerate(goal['message']):
            mes = mes.replace('<span class=\'emphasis\'>', '')
            mes = mes.replace('</span>', '')
            goal['message'][i] = mes

    print("[Goals]")
    user_manual = None
    for key, value in goal.items():
        if not value:           # empty
            continue
        elif key == 'message':  # user manual
            user_manual = value
        else:                   # valid domain
            domain = key        
            print(indent(pformat({domain : value}), ' '*4))
    pprint_manual(user_manual, name)
    

def get_valid_domains(goal):
    """return valid domains for pretty print"""
    domains = []
    for key, value in goal.items():
        if not value:           # empty
            continue
        elif key == 'message':  # user manual
            continue
        else:                   # valid domain
            domains.append(key)
    return domains


def pprint_turns(log, domains):
    """pretty print for dialogue turns"""
    
    # signal for stopping print
    signal = None
    
    for i, log_one in enumerate(log):
        
        # dummy input function for pausing
        print('-' * 20 + '1. Enter to keep going 2. Type \'stop\' and Enter to stop printing ' + '-' * 40)
        signal = input()
        if 'stop' in signal:
            break

        # check whether system turn or not
        bool_sys_turn = False
        if log_one['metadata']:
            bool_sys_turn = True

        # delete span_info
        if 'span_info' in log_one:
            del log_one['span_info']

        # delete unnecessary domains
        domain_pairs = log_one['metadata']
        del_domains = []
        for dom, _ in domain_pairs.items():
            if not dom in domains:
                del_domains.append(dom)
        for dom in del_domains:
            del domain_pairs[dom]
    
        # pretty print
        if bool_sys_turn: print("[SYS]", end=" ")
        else:             print("[USR]", end=" ")
        print("(turn {})".format(i))

        log_one['1. dialogue_state'] = log_one['metadata']
        log_one['2. dialogue_act'] = log_one['dialog_act']
        log_one['3. text'] = log_one['text']
        del log_one['metadata']
        del log_one['dialog_act']
        del log_one['text']
        print(indent(pformat(log_one, width=100), ' ' * 4))
    
    # transform signal to boolean
    if 'stop' in signal:
        signal = True
    else: 
        signal = False
    return signal

### Step 1.1 MultiWOZ 데이터셋을 불러옵니다.

MultiWOZ 데이터셋은 7개의 domain ('hotel', 'train', 'attraction', 'restaurant' 'taxi', 'policy', 'hospital') 으로 구성되어 있으며, 여행정보를 얻고자 하는 'user' 와 이를 도와주는 'system'이 나누는 대화에 대한 데이터 셋입니다.
약 10,000개의 대화 뭉치로 구성되어 있으며, train, validation, test용으로 구분되어 있습니다.

아래 코드를 실행하면, MultiWOZ 데이터 셋 내 train용 데이터 이름 100개가 출력 됩니다.

In [ ]:
cur_dir = os.path.abspath(os.curdir)
print("current directory :", cur_dir)
data_dir = "ConvLab-2/data/multiwoz"
processed_data_dir = os.path.join(cur_dir, 'multiwoz_data/all_data')
if not os.path.exists(processed_data_dir):
    os.makedirs(processed_data_dir)

data_key = ['train', 'val', 'test']
data = {}
for key in data_key:
    data[key] = read_zipped_json(os.path.join(data_dir, key + '.json.zip'), key + '.json')
    print('load {}.json...! '.format(os.path.join(data_dir, key)))
    print('number of dialogues : {}'.format(len(data[key])))
print()

# print available dialogue name until 100
print(list(data['train'].keys())[:100])


## Step 1.2 데이터가 어떻게 생겼는지 살펴봅시다.

위의 출력된 데이터 이름들 중 몇개를 파이썬 리스트 안에 삽입하여 dialogue 를 확인할 수 있습니다. (ex. names = \['SNG0943', 'MUL1801'] ))

한 dialogue는 크게

1. user의 goal, (코드 상에서 'goal'))

2. system과 user의 대화, (코드 상에서 'dialogue_turns'))

로 구분 됩니다.

***

user (\[USR]])는 정의된 goal 및 manual을 읽고, 해당 goal을 달성하기 위해 대화를 수행합니다.

system (\[SYS])은 user의 goal을 알지 못하고, 대화를 통해 (1)user가 원하는 조건을 파악하고, (2)user가 원하는 정보를 제공하며, (3)필요시 예약을 수행합니다.

***

goal 내에서,

`info`는 user 입장에서, system에게 user가 원하는 조건 및 니즈를 알려주고자(inform) 하는 내용이고,

`reqt`는 user 입장에서, system에게 uesr가 요청하고자(request) 하는 내용입니다.

***

본 데이터셋의 경우 __system model을 만드는 것__을 목표로 합니다.

***

아래 코드를 실행하면 goal 및 발화를 살펴볼 수 있으며, Enter로 넘길 수 있습니다. 

그만 보고 싶다면 stop을 입력 후 Enter 합니다.

In [ ]:
# You can handle dialogue_names
dialogue_names = ['SNG0943', 'MUL1801']

for name in dialogue_names:
    
    print()
    print('-' * 125)
    print("[Dialogue name] \'{}\'".format(name))

    # access datum using name
    datum = data['train'][name]
    goal = datum['goal']
    dialogue_turns = datum['log']

    # print goal and dialogue turns
    pprint_goal(goal, name)
    valid_domains = get_valid_domains(goal)
    break_signal = pprint_turns(dialogue_turns, valid_domains)  # If you don't want to see print, please comment!
    # break_signal = pprint_turns(dialogue_turns, valid_domains)    Like this!

    if break_signal:
        break

# Step 2. ConvLab을 활용해 Pipelined 대화 시스템을 구축하자

## Step 2.0 필요한 module들을 정의합니다

아래 코드를 도와주는 module들 이며, 필요시 수정할 수 있습니다.

In [ ]:
# common import: convlab2.$module.$model.$dataset
from convlab2.nlu.jointBERT.multiwoz import BERTNLU
from convlab2.nlu.milu.multiwoz import MILU
from convlab2.dst.rule.multiwoz import RuleDST
from convlab2.policy.rule.multiwoz import RulePolicy
from convlab2.nlg.template.multiwoz import TemplateNLG
from convlab2.dialog_agent import BiSession, Agent # , # PipelineAgent
from convlab2.evaluator.multiwoz_eval import MultiWozEvaluator
from pprint import pprint
import random
import numpy as np
import torch
import spacy

import logging 
# uncessary logging block
mpl_logger = logging.getLogger('matplotlib') 
mpl_logger.setLevel(logging.WARNING) 
cntp_logger = logging.getLogger('urllib3.connectionpool')
cntp_logger.setLevel(logging.WARNING)
ttu_logger = logging.getLogger('transformers.tokenization_utils')
ttu_logger.setLevel(logging.WARNING)
tcu_logger = logging.getLogger('transformers.configuration_utils')
tcu_logger.setLevel(logging.WARNING)
tmu_logger = logging.getLogger('transformers.modeling_utils')
tmu_logger.setLevel(logging.WARNING)
logging.getLogger().setLevel(logging.INFO)
import warnings
warnings.filterwarnings("ignore")

spacy.cli.download('en_core_web_sm')
spacy.load('en_core_web_sm')

from convlab2.nlu import NLU
from convlab2.dst import DST
from convlab2.policy import Policy
from convlab2.nlg import NLG
from copy import deepcopy

class PipelineAgent(Agent):
    """Pipeline dialog agent base class, including NLU, DST, Policy and NLG.

    The combination modes of pipeline agent modules are flexible. The only thing you have to make sure is that
    the API of agents are matched.

    Example:
        If agent A is (nlu, tracker, policy), then the agent B should be like (tracker, policy, nlg) to ensure API
        matching.
    The valid module combinations are as follows:
           =====   =====    ======  ===     ==      ===
            NLU     DST     Policy  NLG     In      Out
           =====   =====    ======  ===     ==      ===
            \+      \+        \+    \+      nl      nl
             o      \+        \+    \+      da      nl
             o      \+        \+     o      da      da
            \+      \+        \+     o      nl      da
             o       o        \+     o      da      da
           =====   =====    ======  ===     ==      ===
    """

    def __init__(self, nlu: NLU, dst: DST, policy: Policy, nlg: NLG, name: str):
        """The constructor of PipelineAgent class.

        Here are some special combination cases:

            1. If you use word-level DST (such as Neural Belief Tracker), you should set the nlu_model paramater \
             to None. The agent will combine the modules automitically.

            2. If you want to aggregate DST and Policy as a single module, set tracker to None.

        Args:
            nlu (NLU):
                The natural langauge understanding module of agent.

            dst (DST):
                The dialog state tracker of agent.

            policy (Policy):
                The dialog policy module of agent.

            nlg (NLG):
                The natural langauge generator module of agent.
        """
        super(PipelineAgent, self).__init__(name=name)
        assert self.name in ['user', 'sys']
        self.opponent_name = 'user' if self.name is 'sys' else 'sys'
        self.nlu = nlu
        self.dst = dst
        self.policy = policy
        self.nlg = nlg
        self.init_session()
        self.history = []

        self.print_nlu = False
        self.print_dst = False
        self.print_pol = False

    def response(self, observation, print_nlu=False, print_dst=False, print_pol=False):
        """Generate agent response using the agent modules."""
        # Note: If you modify the logic of this function, please ensure that it is consistent with deploy.server.ServerCtrl._turn()

        self.print_nlu = print_nlu
        self.print_dst = print_dst
        self.print_pol = print_pol

        if self.dst is not None:
            self.dst.state['history'].append([self.opponent_name, observation]) # [['sys', sys_utt], ['user', user_utt],...]
        self.history.append([self.opponent_name, observation])
        # get dialog act
        if self.nlu is not None:
            self.input_action = self.nlu.predict(observation, context=[x[1] for x in self.history[:-1]])
        else:
            self.input_action = observation
        self.input_action = deepcopy(self.input_action) # get rid of reference problem
        if self.print_nlu:
            print("nlu predict")
            pprint(self.input_action)
        # get state
        if self.dst is not None:
            if self.name is 'sys':
                self.dst.state['user_action'] = self.input_action
            else:
                self.dst.state['system_action'] = self.input_action
            state = self.dst.update(self.input_action)
        else:
            state = self.input_action
        state = deepcopy(state) # get rid of reference problem
        if self.print_dst:
            print("dialogue state predict")
            pprint({'dialogue state': state['belief_state'], 'history': state['history']})
        # get action
        self.output_action = deepcopy(self.policy.predict(state)) # get rid of reference problem
        if self.print_pol:
            print("dialogue act predict")
            pprint(self.output_action)

        # get model response
        if self.nlg is not None:
            model_response = self.nlg.generate(self.output_action)
        else:
            model_response = self.output_action
        # print("model response {}".format(model_response))
        if self.dst is not None:
            self.dst.state['history'].append([self.name, model_response])
            if self.name is 'sys':
                self.dst.state['system_action'] = self.output_action
            else:
                self.dst.state['user_action'] = self.output_action
        self.history.append([self.name, model_response])
        return model_response

    def is_terminated(self):
        if hasattr(self.policy, 'is_terminated'):
            return self.policy.is_terminated()
        return None

    def get_reward(self):
        if hasattr(self.policy, 'get_reward'):
            return self.policy.get_reward()
        return None

    def init_session(self):
        """Init the attributes of DST and Policy module."""
        if self.nlu is not None:
            self.nlu.init_session()
        if self.dst is not None:
            self.dst.init_session()
            if self.name == 'sys':
                self.dst.state['history'].append([self.name, 'null'])
        if self.policy is not None:
            self.policy.init_session()
        if self.nlg is not None:
            self.nlg.init_session()
        self.history = []

    def get_in_da(self):
        return self.input_action

    def get_out_da(self):
        return self.output_action


## Step 2.1 pipelined 대화 시스템 예시를 살펴봅시다.

우선, system model을 pipelined 대화 시스템으로 구성했을 때, 대화를 어떻게 수행할 수 있는지 살펴봅시다.

pipelined 대화 모델은 크게 4가지로 구성되어 있습니다.

NLU (Natural Language Understanding (언어 인식 모듈)) : 직전의 상대방 발화를 이해 및 해석 합니다.

DST (Dialogue State Tracking (대화 상태 추적 모듈)) : 현재까지 대화의 맥락을 추적하고, 변경 사항을 업데이트 합니다.

Dialogue Policy (대화 정책 모듈) : 다음 발화를 위해 구조화된 단어 형태로 정책을 결정 합니다. (자연스러운 문장의 형태가 아닌, 의도만을 결정합니다.)

NLG (Natural Language Generation (언어 생성 모듈)) : 결정된 정책을 가지고, 사람이 이해할 수 있는 자연어를 생성합니다.

-----------------

아래는 가장 기본적인 Pipelined 대화 시스템을 구성한 예시입니다.

BERT NLU : 앞선 practice 1에서 다룬 BERT NLU

RuleDST : Rule 기반 DST module

RulePolicy : Rule 기반 Policy module

TemplateNLG : Template 기반 (정해진 템플릿 위에서 단어를 채워넣는 방식) NLG module

4가지 모듈에 대해 정의를 끝마쳤다면, PipelineAgent라는 wrapper 에 씌워 sys_agent를 선언합니다.

In [ ]:
# MILU
sys_nlu = MILU()
# simple rule DST
sys_dst = RuleDST()
# rule policy
sys_policy = RulePolicy()
# template NLG
sys_nlg = TemplateNLG(is_user=False)
# assemble
sys_agent = PipelineAgent(sys_nlu, sys_dst, sys_policy, sys_nlg, name='sys')

sys_agent.response("user의 발화", print_nlu=False, print_dst=False, print_pol=False) 를 하면, user의 발화에 대한 응답을 합니다.

print_nlu, print_dst, print_pol을 True로 변경하면, 해당하는 value를 print 해볼 수 있습니다.

In [ ]:
sys_agent.init_session()
sys_agent.response("I want to find a moderate hotel", print_nlu=False, print_dst=False, print_pol=False)

In [ ]:
sys_agent.response("Which type of hotel is it ?")

In [ ]:
sys_agent.response("OK , where is its address ?")

In [ ]:
sys_agent.response("Thank you !")

In [ ]:
sys_agent.response("Try to find me a Chinese restaurant in south area .")

In [ ]:
sys_agent.response("Which kind of food it provides ?")

In [ ]:
sys_agent.response("Book a table for 5 , this Sunday .")

## Step 2.2 system agent와 대화할 user simulator를 구성해봅시다.

system model의 성능을 알아보기 위해서는 user 역할을 할 수 있는 user simulator가 필요합니다. 

사람이 매번 user의 역할을 하여 대화를 주고 받는 것은 많은 노동력을 필요로 하기 때문입니다. 

특히, Dialog Policy를 RL agent로 두었을 때, 다양한 대화 시도를 위해서 user simulator 는 필수적입니다.

ConvLab에서는 RulePolicy(character='usr')로 두었을 때, `Agenda` policy로 변환되며, 이는 user의 goal을 기반으로 하는 user model을 정의할 수 있습니다. 

또한, `Agenda` policy는 dst 모델까지 함께 포함하고 있기 때문에 `user_dst = None` 이 됩니다.


In [ ]:
# MILU
user_nlu = MILU()
# not use dst
user_dst = None
# rule policy
user_policy = RulePolicy(character='usr')   # UserPolicyAgendaMultiWoz()
# template NLG
user_nlg = TemplateNLG(is_user=True)
# user_nlg = SCLSTM(is_user=True)
# assemble
user_agent = PipelineAgent(user_nlu, user_dst, user_policy, user_nlg, name='user')

## Step 2.3 user simulator 와 system model 간 대화를 수행해봅시다.

지금까지, 우리는 user simulator와 system model을 정의 했습니다.

`MultiWozEvaluator` 클래스는 성능을 평가하기 위해 사용됩니다. (user의 goal을 정의해줍니다.)

`BiSession` 클래스는 user simulator와 system model의 대화 및 평가를 도와줍니다.

`next_turn` 함수는 한 턴의 대화를 수행합니다.

### 평가 지표

success rate : 예약 성사 + recall == 1, 즉 유저의 조건에 부합하는 예약을 해내고, 물어보는 모든 정보에 대해 알맞은 갑ㅅ을 출력

book rate : 예약 성사율 (= 예약 성공 수 / 예약 정답 수) 

Inform precision : (TP) / (TP + FP), precision이 낮다는 것은 요청한 slot 외 불필요한 정보를 많이 알려주는 것으로 해석할 수 있음.

Inform recall : (TP) / (TP + FN), recall이 낮다는 것은 요청한 slot에 대해 대답하지 못한 것으로 해석 할 수 있음.

Inform F1 : Precision & Recall에 대한 조화 평균



In [ ]:
def set_seed(r_seed):
    random.seed(r_seed)
    np.random.seed(r_seed)
    torch.manual_seed(r_seed)

evaluator = MultiWozEvaluator()
sess = BiSession(sys_agent=sys_agent, user_agent=user_agent, kb_query=None, evaluator=evaluator)

set_seed(20200804)

sys_response = ''
sess.init_session()
print('init goal:')
pprint(sess.evaluator.goal)
print('-'*100)
for i in range(20):
    sys_response, user_response, session_over, reward = sess.next_turn(sys_response)
    print('user:', user_response)
    print('sys:', sys_response)
    print()
    if session_over is True:
        break
print('task success:', sess.evaluator.task_success())
print('book rate:', sess.evaluator.book_rate())
print('inform precision/recall/f1:', sess.evaluator.inform_F1())
print('-'*50)
print('final goal:')
pprint(sess.evaluator.goal)

print('='*100)

# Step 3. ConvLab에서 제공하는 모듈들로 모델들을 구성 및 진단하고, 평가하자

## Step 3.0. ConvLab 에서 지원하는 모델들을 load 합니다.

이용가능한 model들:

- NLU: BERTNLU, MILU, SVMNLU
- DST: RuleDST
- Word-DST: SUMBT, TRADE (set `sys_nlu` to `None`)
- Policy: RulePolicy, Imitation, REINFORCE, PPO, GDPL
- Word-Policy: MDRG, HDSA, LaRL (set `sys_nlg` to `None`)
- NLG: Template, SCLSTM
- End2End: Sequicity, DAMD, RNN_rollout (directly used as `sys_agent`)
- Simulator policy: Agenda, VHUS (for `user_policy`)


In [ ]:
# available NLU models
from convlab2.nlu.svm.multiwoz import SVMNLU
from convlab2.nlu.jointBERT.multiwoz import BERTNLU
from convlab2.nlu.milu.multiwoz import MILU
# available DST models
from convlab2.dst.rule.multiwoz import RuleDST
#from convlab2.dst.mdbt.multiwoz import MDBT
from convlab2.dst.sumbt.multiwoz import SUMBT
from convlab2.dst.trade.multiwoz import TRADE
# available Policy models
from convlab2.policy.rule.multiwoz import RulePolicy
from convlab2.policy.ppo.multiwoz import PPOPolicy
from convlab2.policy.pg.multiwoz import PGPolicy
from convlab2.policy.mle.multiwoz import MLEPolicy
from convlab2.policy.gdpl.multiwoz import GDPLPolicy
#from convlab2.policy.vhus.multiwoz import UserPolicyVHUS
from convlab2.policy.mdrg.multiwoz import MDRGWordPolicy
from convlab2.policy.hdsa.multiwoz import HDSA
from convlab2.policy.larl.multiwoz import LaRL
# available NLG models
from convlab2.nlg.template.multiwoz import TemplateNLG
from convlab2.nlg.sclstm.multiwoz import SCLSTM
# available E2E models
from convlab2.e2e.sequicity.multiwoz import Sequicity
from convlab2.e2e.damd.multiwoz import Damd

## Step 3.1. ConvLab에서 지원하는 모델들을 가지고 나만의 대화시스템을 만들어 봅시다.

Word-DST 모델들은 NLU와 DST가 합쳐진 모델을 의미합니다. 따라서, 별도의 NLU 모델 없이 사용할 수 있습니다.

따라서, (1) NLU+RuleDST 또는 (2) Word-DST 로 조합이 가능합니다.

[주의!] Word-DST 의 경우 sys_nlu = None 이어야 합니다.

Word-Policy 모델들은 Dialogue Policy 와 NLG 가 합쳐진 모델을 의미합니다. 따라서 별도의 NLG 모델없이 사용할 수 있습니다.

따라서, (1) Policy+NLG 또는 Word-Policy 로 조합이 가능합니다.

[주의!] Word-Policy 의 경우 sys_nlg = None 이어야 합니다.

`PipelineAgent` class를 이용해 Pipelined 대화 시스템을 만들 수 있습니다. 또는 End-to-End model를 사용할 수도 있습니다.



In [ ]:
# NLU+RuleDST:
sys_nlu = MILU()
# sys_nlu = SVMNLU()
# sys_nlu = BERTNLU()
sys_dst = RuleDST()

# or Word-DST:
# sys_nlu = None
# sys_dst = SUMBT()
# sys_dst = TRADE()
#### (not working!) sys_dst = MDBT()

# [Caution] In Word-DST case, sys_nlu must be "None"

# Policy+NLG:
sys_policy = RulePolicy()
# sys_policy = PPOPolicy()
# sys_policy = PGPolicy()
# sys_policy = MLEPolicy()
# sys_policy = GDPLPolicy()
sys_nlg = TemplateNLG(is_user=False)
#sys_nlg = SCLSTM(is_user=False)

# or Word-Policy:
# sys_policy = LaRL()
# sys_policy = HDSA()
# sys_policy = MDRGWordPolicy()
# sys_nlg = None

# [Caution] "In Word-policy case, sys_nlg must be None"

sys_agent = PipelineAgent(sys_nlu, sys_dst, sys_policy, sys_nlg, 'sys')
# sys_agent = Sequicity()
# sys_agent = Damd()

앞에서 했던 방식대로, user simulator도 정의해줍니다.

(ConvLab에서는 RulePolicy(character='usr')로 두었을 때, `Agenda` policy로 변환되며, 이는 user의 goal을 기반으로 하는 user model을 정의할 수 있습니다. )

In [ ]:
user_nlu = BERTNLU()
# user_nlu = MILU()
# user_nlu = SVMNLU()
user_dst = None
user_policy = RulePolicy(character='usr')
# user_policy = UserPolicyVHUS(load_from_zip=True)
user_nlg = TemplateNLG(is_user=True)
# user_nlg = SCLSTM(is_user=True)
user_agent = PipelineAgent(user_nlu, user_dst, user_policy, user_nlg, name='user')

## Step 3.2 분석 툴을 사용해 system model을 진단해봅시다.

Convlab2에서는 분석 툴(analysis tool)을 제공하며, 이를 통해 정의한 system model의 성능 및 취약점을 분석하고 진단할 수 있습니다.

뿐만 아니라 HTML report를 작성해주어, 조금 더 풍부한 통계 정보를 얻어낼 수 있습니다. (results/\$model_name\$ 를 참조합니다)

In [ ]:
from convlab2.util.analysis_tool.analyzer import Analyzer

# if sys_nlu!=None, set use_nlu=True to collect more information
analyzer = Analyzer(user_agent=user_agent, dataset='multiwoz')

set_seed(20200131)
analyzer.comprehensive_analyze(sys_agent=sys_agent, model_name='sys_agent', total_dialog=20)

## Step 3.3 여러 개의 system model 간 성능을 비교해봅시다. 

서로 다른 3가지의 system model에 대한 결과를 아래에 채워봅시다. (vscode 기준, 더블 클릭하면 수정할 수 있습니다.)

NLU       | DST       | Policy    | NLG       | Success rate | Book rate | Inform P | Inform R | Inform F1 | Turn(succ/all) |
--------- | --------- | --------- | --------- | :----------: | :-------: | -------- | -------- | --------- | -------------- |
Content   | Content   | Content   | Content   | Content      | Content   | Content  | Content  | Content   | Content        |
Content   | Content   | Content   | Content   | Content      | Content   | Content  | Content  | Content   | Content        |
Content   | Content   | Content   | Content   | Content      | Content   | Content  | Content  | Content   | Content        |

In [ ]:
set_seed(20200805)

# define your own system agent2
# sys_agent2 = PipelineAgent(...)

# define your own system agent3
# sys_agent3 = PipelineAgent(...)

analyzer.compare_models(agent_list=[sys_agent, sys_agent, sys_agent], model_name=['sys_agent1', 'sys_agent2', 'sys_agent3'], total_dialog=100)

# Additional. End-to-end Neural Pipeline (ACL 2020) 모델을 사용해보자

Paper : Donghoon Ham *, Jeong-Gwan Lee *, Youngsoo Jang, and Kee-Eung Kim. 2020. End-to-End Neural Pipeline for Goal-Oriented Dialogue System using GPT-2. ACL 2020

![Model architecture](image/e2e_model.png)

우선, Convlab2에 있는 모델을 import 하고 multiwoz로 pretrained된 weight를 다운로드 합니다.

In [ ]:
from convlab2.e2e.Transformer import Transformer
sys_agent = Transformer()

위에서 사용했던 다른 e2e agent 와는 다르게, neural pipeline 모델은 dialogue state, system action (dialogue policy) 을 확인할 수 있습니다.

In [ ]:
sys_agent.init_session()
sys_agent.response("I want to find a moderate hotel")

## Neural pipeline 모델과 대화해보고, 성능을 평가해봅시다 !

In [ ]:
sys_agent.init_session()
while True:
    raw_text = input(">>> ")
    while not raw_text:
        print('not empty')
        raw_text = input(">>> ")
    if raw_text == 'r':
        sys_agent.init_session()
        continue
    out_text = sys_agent.response(raw_text)
    print('sys: ', out_text)

In [ ]:
analyzer = Analyzer(user_agent=user_agent, dataset='multiwoz')
set_seed(20200131)
analyzer.comprehensive_analyze(sys_agent=sys_agent, model_name='sys_agent', total_dialog=20)